In [2]:
import pandas as pd
import numpy as np

In [3]:
glasses = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/glasses.csv')

In [4]:
glasses['accel_x'] = glasses['ACC_X']
glasses['accel_y'] = glasses['ACC_Y']
glasses['accel_z'] = glasses['ACC_Z']
glasses['gyro_x'] = glasses['GYRO_X']
glasses['gyro_y'] = glasses['GYRO_Y']
glasses['gyro_z'] = glasses['GYRO_Z']

In [5]:
glasses = glasses.drop(columns= ['NUM', 'EOG_L', 'EOG_R', 'EOG_H', 'EOG_V', 'ACC_Z', 'ACC_X', 'ACC_Y', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Exclude the 'timestamp' column from standardization
cols_to_scale = [col for col in glasses.columns if col != 'timestamp']
glasses[cols_to_scale] = scaler.fit_transform(glasses[cols_to_scale])

def complementary_filter(glasses, alpha):
    dt = 0.02  # Assuming a sampling rate of 50 Hz (1/50 = 0.02)
    gyro = glasses[['gyro_x', 'gyro_y', 'gyro_z']].values  # Extract gyro data
    acc = glasses[['accel_x', 'accel_y', 'accel_z']].values  # Extract accelerometer data

    roll_acc = np.arctan2(acc[:, 1], acc[:, 2])  # Calculate roll from accelerometer data
    pitch_acc = np.arctan2(-acc[:, 0], np.sqrt(acc[:, 1] ** 2 + acc[:, 2] ** 2))  # Calculate pitch from accelerometer data

    roll = np.zeros_like(roll_acc)
    pitch = np.zeros_like(pitch_acc)
    yaw = np.zeros_like(gyro[:, 0])

    for i in range(len(acc)):
        roll[i] = alpha * (roll[i] + gyro[i, 0] * dt) + (1 - alpha) * roll_acc[i]
        pitch[i] = alpha * (pitch[i] + gyro[i, 1] * dt) + (1 - alpha) * pitch_acc[i]
        yaw[i] = gyro[i, 2]  # Yaw directly from gyroscope data

    return np.rad2deg(roll), np.rad2deg(pitch), np.rad2deg(yaw)

alpha = 0.98  # Adjust alpha as needed
roll, pitch, yaw = complementary_filter(glasses, alpha)

# Add Euler angles as new columns to the DataFrame
glasses['orientation_x'] = roll
glasses['orientation_y'] = pitch
glasses['orientation_z'] = yaw

In [ ]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 283.615 or orientation_z < -283.24:
                motion_type = "Vertical Movement"
            elif orientation_x > 12.04 or orientation_x < -11.25:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
glasses['motion_type'] = glasses.apply(classify_motion_row, axis=1)